# Setup

If you run this notebook in Colab, please change **drive_path** to the path leading to the directory on your Google Drive in which `smplx/` is stored.

In [ ]:
### Mount google drive if available
try:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/master_thesis/'
    in_colab = True
except:
    drive_path = ''
    in_colab = False

IGNORE the cell below if the repo has already been cloned and the dependencies installed. If you run this notebook in Colab, RUN the cell.

In [ ]:
### Clone repository and install dependencies

!git clone https://github.com/maximeraafat/humbi_textured_meshes.git
!pip install -r humbi_textured_meshes/requirements.txt
%cd humbi_textured_meshes

In [ ]:
# @title Provide the HUMBI data root URL

# @markdown Proceed to the [HUMBI website](https://humbi-data.net), register and obtain the data root URL (https*******.amazonaws.com), and modify the **HUMBI_ROOT_URL** variable below.

HUMBI_ROOT_URL = '' # @param {type:"string"}

file_object = open('humbi_root_url.txt', 'a')
file_object.write(HUMBI_ROOT_URL)
file_object.close()

In [ ]:
### Install remaining dependencies (can take a few minutes)

# pytorch3d
import os
import sys
import torch
import requests

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True

if need_pytorch3d:
    if torch.__version__.startswith("1.11.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


# detectron2
need_detectron=False
try:
    import detectron2
except ModuleNotFoundError:
    need_detectron=True   

if need_detectron:
    !git clone https://github.com/facebookresearch/detectron2.git detectron2_repo
    torch_version = ".".join(torch.__version__.split(".")[:2])
    cuda_version = torch.__version__.split("+")[-1]
    url = 'https://dl.fbaipublicfiles.com/detectron2/wheels/%s/torch%s/index.html' % (cuda_version, torch_version)
    try:
        # We try to install Detectron2 via a released wheel.
        requests.get(url).raise_for_status()
        !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
    except:
        # We try to install Detectron2 from source.
        !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


# cleanup
!rm -rf 1.10.0.tar.gz cub-1.10.0/

# Mesh reconstruction and UV color texture learning

Optional flags

1. `--subject $SUBJECTS` : list or range of subjects for which we construct a textured mesh. SUBJECTS is a list passed as a string, e.g. `"[1,2,3]"` or `"range(1,10)"`. If not passed, all HUMBI subjects are constructed, i.e., `"range(1, 618)"`

2. `--poses $POSES` : list of poses (same length as SUBJECTS!). If e.g. `SUBJECTS = "[1,2,3]"` and `POSES = "[1, 9, 25]"`, then subject **1** will be reconstructed for pose **00000001**, subject **2** in pose **00000009** and subject **3** in pose **00000025** If not passed, each subject will be reconstructed in the default T-pose

3. `--gdrive $drive_path` : path leading to the directory in which `smplx/` is stored, and in which the constructed textured meshes will be stored. By default it is set to the current working directory

4. `--iters 30` : number of training epochs (one epoch = one iteration through all available cameras per subject in the considered pose)

5. `--saveobj` : whether to store the training progress in an `.obj` file every 3 epochs (stored under `<drive_path>/humbi_output/humbi_smplx_objs`)

6. `--smoothing` : whether to slightly smooth the reconstructed mesh after learning the vertex displacements

7. `--nodisps` : disable storing the displacement maps (only rgb textures will be saved). By default, rgb and displacement will both be stored in 2 different texture maps

8. `--val` : whether to perform validation on the 10% of the data (and leave 10% out for testing)

In [ ]:
### Textured deformed SMPL-X mesh reconstruction

# SUBJECTS is called as a string, and $SUBJECTS evaluates a string, therefore we need SUBJECTS to be a string within a string
SUBJECTS = " '[1, 70, 122]' "

if in_colab:
    %run main.py --subjects $SUBJECTS --gdrive $drive_path --iters 4 --saveobj --smoothing
else:
    %run main.py --subjects $SUBJECTS --iters 4 --saveobj --smoothing

Notice that the displacement textures stored under `<drive_path>/humbi_output/humbi_smplx_geom` are normalized, i.e., the color values are saturated to take advantage of all pixel intensities (or bits).

In order to revert the normalization process and get the true displacement textures, execute the cells in the next section.

# Get true displacement textures via inverse normalizaton

The true displacement textures will be stored for the selected subjects in `<drive_path>/humbi_output/humbi_smplx_true_geom/`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils.normalize_disps import denormalize_disp

In [ ]:
### Inverse normalization on displacement maps

SUBJECTS = [1, 70, 122]
save_path_disps = drive_path + 'humbi_output/humbi_smplx_true_geom/'

os.makedirs(save_path_disps, exist_ok=True)
for subject in SUBJECTS:

    img_path = drive_path + 'humbi_output/humbi_smplx_geom/disp_texture_%d.png' % subject

    normalization = np.load(drive_path + 'humbi_output/humbi_smplx_npz/normalization.npz')
    global_min = normalization['global_min'].item()
    global_max = normalization['global_max'].item()

    image = denormalize_disp(img_path, global_min, global_max)

    plt.figure(figsize=(10, 10))
    plt.imshow(image + 0.5)
    plt.axis("off")
    plt.title('subject %d\n' % subject)
    plt.imsave(save_path_disps + 'disp_texture_%d.png' % subject, image + 0.5)